# Manejo de expresiones regulares para extraer información simple

In [2]:
# Para importar el módulo con las funciones necesarias
import sys
sys.path.append(r"D:\Repos Github\AnaliticaTextual-Ejemplos") # la carpeta donde está el módulo

import utils # el módulo -> utils.py 

In [3]:
import es_core_news_sm # procesador de lenguaje con segmentación de frases, tokenización, lematización, etiquetas POS y dependencias sintácticas. Carga el modelo en español de Spacy
import regex # sirve para usar una librería de expresiones regulares más potente que la librería estándar 're' de Python.

In [4]:
def BusquedaSimple(texto):
    r = regex.findall('[0-9]+',texto) # tiene que buscar en el texto que le pasemos una secuencia de  1 o más caracteres (+) de 0 a 9. Esto capta todas las coincidencias en el texto
    return(r)

In [5]:
def ExtraerNombresDocumento(Lineas):
   lista = []
   patron = r'(\w+)/(PROPN|NOUN)' # Una o más letras (\w+) con una barra inclinada (/) etiquetas de Nombres Propios (PROPN) O (|) Nombres comunes (NOUN)
   for linea in Lineas:
      ListaPalabras  = ExtraerNombresLinea(patron,linea)
      if ListaPalabras != []:
         lista.append(ListaPalabras)
   return(lista)

In [6]:
def ExtraerNombresLinea(patron,linea):
      texto      = linea.rstrip()
      etiquetado = utils.Etiquetar(texto)    #devuelve la frase POS-taggeada -> "Juan/PROPN vive/VERB en/ADP Madrid/PROPN"
      ListaPalabras   = [w for (w,t) in regex.findall(patron,etiquetado)] #regex.findall devuelve todas las coincidencias del patrón (cada vez que el patrón de expresión regular se encuentra en el texto), como tuplas. ("Juan", "PROPN"), ("perro", "NOUN")
        #recorre cada tupla (w,t), e queda solo con w (la palabra)
      return(ListaPalabras)

In [13]:
def ExtraerNumeros(lineas):
    for linea in lineas:
      linea = linea.rstrip() #elimina espacios o saltos de línea al final
      match = regex.findall("[0-9]+", linea) # "[0-9]+" uno o más digitos seguidos
      if match:
        return(match)
    return(None)

    """
    Objetivo: Recorrer varias líneas, Buscar números en cada una, Devolver los números cuando se encuentren. Si no hay ninguno, devolver None
    """

In [39]:
def BusquedaRelacion(texto):
   
    """
    Busca un único patrón grande que dentro tiene varios grupos (...), un + que repite una parte y varias etiquetas POS encadenadas
    (\s*(\w+)/PROPN)+      # grupo 1 (y dentro grupo 2)
    (\w+)/NOUN             # grupo 3                                   
    (\w+)/PROPN            # grupo 4
    Grupo 1: toda la secuencia de nombres propios (nombre + apellido + …)
    Grupo 2: la última palabra de esa secuencia (por cómo funciona regex)
    Grupo 3: el sustantivo (director, analista, gerente…)
    Grupo 4: el nombre propio final (empresa, ciudad…)
    
    Ejemplo-> Juan/PROPN Pérez/PROPN ,/PUNCT director/NOUN en/ADP Microsoft/PROPN    
    
    (             # ← Grupo 1
    \s*        # espacios opcionales
    (          # ← Grupo 2
       \w+    # palabra (solo ASCII)
    )          
    /PROPN     # etiqueta PROPN
    )+         # ← el conjunto (grupo1) se repite UNA O MÁS VECES

    \s*\,/PUNCT   # coma etiquetada como PUNCT

    \s*(\w+)/NOUN # ← Grupo 3 (sustantivo)

    \s*\w+/ADP    # (preposición)

    \s*(\w+)/PROPN # ← Grupo 4 (nombre propio final)

    
    """
    etiquetado = utils.Etiquetar(texto)
    patron =r'(\s*(\w+)/PROPN)+\s*\,/PUNCT\s*(\w+)/NOUN\s*\w+/ADP\s*(\w+)/PROPN'
    match = regex.search(patron, etiquetado)
    return(match)

In [30]:
FILENAME = r'D:\Repos Github\AnaliticaTextual-Ejemplos\CORPUS\deportes\d1.txt'

In [31]:
texto = "El numéro de contagiados pasó de 1000 a 2000 en una semana"

In [32]:
lista = BusquedaSimple(texto)
print(lista)

['1000', '2000']


In [33]:
nlp = es_core_news_sm.load()

In [36]:
lineas = open(FILENAME, encoding="latin-1")

In [37]:
Nombres = ExtraerNombresDocumento(lineas)
print(Nombres)

[['golfista', 'jornada', 'tarjeta', '1', 'parte', 'certamen', 'Texas', 'Estados', 'Unidos'], ['Joaquín', 'Niemann', 'golfista', 'corte', 'fin', 'semana', 'Charles', 'Schwab', 'Challenge', 'PGA', 'Tour'], ['chileno', 'día', 'parejo', 'Texas', 'rendimiento', 'parte', 'ronda', 'tarjeta', '1'], ['talagantino', 'horas', 'resultados', 'rivales', 'corte', '2', 'competencia'], ['Niemann', 'birdies', 'hoyos', 'bogeys', 'banderas', 'registro', 'participación', 'partir', 'horas', 'Chile'], ['sueco', 'Jonas', 'Blixt', 'líder', '9', 'birdies', 'hoyos', 'águila', 'bandera']]


In [40]:
rel = BusquedaRelacion("Maria Barrientos Tapia, asesora del Ministerio")
if rel is not None:
    print(rel.captures(2))
    print(rel.captures(3))
    print(rel.captures(4))


# la librería regex tiene una función especial: .captures(n), que devuelve TODAS las capturas del grupo n si ese grupo aparece varias veces.

['Maria', 'Barrientos', 'Tapia']
['asesora']
['Ministerio']


Importante

search() → busca la primera coincidencia

match() → busca solo al principio

findall() → devuelve todas las coincidencias

finditer() → devuelve un iterador sobre todas las coincidencias